In [ ]:
import os
from pathlib import Path

import numpy as np
from openslide import OpenSlide
from PIL import Image
from torchvision.datasets.folder import default_loader

In [ ]:
import staintools

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from violet.utils.preprocessing import extract_svs_tiles, get_he_image, extract_tif_tiles, get_svs_array, normalize_he_image
from violet.utils.dataloaders import listfiles

In [ ]:
fps = [
    '/home/estorrs/violet/data/he/ffpe/tcia_pda/train/C3L-00017-21/C3L-00017-21_100_104.jpeg',
    '/home/estorrs/violet/data/he/ffpe/tcia_pda/train/C3L-00277-26/C3L-00277-26_100_120.jpeg',
    '/home/estorrs/violet/data/he/ffpe/tcia_pda/train/C3L-00189-22/C3L-00189-22_100_539.jpeg',
    '/home/estorrs/violet/data/he/ffpe/tcia_ccrcc/train/C3L-00011-21/C3L-00011-21_100_37.jpeg',
    '/home/estorrs/violet/data/he/ffpe/tcia_ccrcc/train/C3L-00165-22/C3L-00165-22_105_66.jpeg',
    '/home/estorrs/violet/data/st/ccrcc_ffpe_08032021/train/HT282N1_S1H3Fs4U1/HT282N1_S1H3Fs4U1_AAACTCGTGATATAAG-1.jpeg',
    '/home/estorrs/violet/data/st/ccrcc_ffpe_08032021/train/HT293N1_S1H3Fs1U1/HT293N1_S1H3Fs1U1_AACAATACATTGTCGA-1.jpeg',
    '/home/estorrs/violet/data/st/pdac_ffpe_07292021/train/HT270P1_S1H1Fs5U1/HT270P1_S1H1Fs5U1_AAACTCGGTTCGCAAT-1.jpeg',
    '/home/estorrs/violet/data/st/pdac_ffpe_07292021/train/HT270P1_S1H1Fs5U1/HT270P1_S1H1Fs5U1_AGACAGGCATCTCAGC-1.jpeg'
]

In [ ]:
for fp in fps:
    print(fp)
    im = default_loader(fp)
    im.show()

In [ ]:
ref = '/home/estorrs/violet/data/he/ffpe/tcia_ccrcc/train/C3L-00165-22/C3L-00165-22_105_66.jpeg'
# ref = '/home/estorrs/violet/data/he/ffpe/tcia_pda/train/C3L-00017-21/C3L-00017-21_100_104.jpeg'
target = staintools.read_image(ref)
# Standardize brightness (optional, can improve the tissue mask calculation)
target = staintools.LuminosityStandardizer.standardize(target)

# Stain normalize
normalizer = staintools.StainNormalizer(method='vahadane')
normalizer.fit(target)

In [ ]:
import matplotlib.pyplot as plt
for fp in fps:
    print(fp)
    im = default_loader(fp)
    im.show()
    
    to_transform = staintools.read_image(fp)
    to_transform = staintools.LuminosityStandardizer.standardize(to_transform)
    transformed = normalizer.transform(to_transform)
    plt.imshow(transformed)
    plt.show()



In [ ]:
import matplotlib.pyplot as plt
for fp in fps:
    print(fp)
    im = default_loader(fp)
    im.show()
    
    to_transform = staintools.read_image(fp)
    to_transform = staintools.LuminosityStandardizer.standardize(to_transform)
    transformed = normalizer.transform(to_transform)
    plt.imshow(transformed)
    plt.show()



In [ ]:
%%time
for fp in fps:
    print('a')
    to_transform = staintools.read_image(fp)
    print('b')
    to_transform = staintools.LuminosityStandardizer.standardize(to_transform)
    print('c')
    transformed = normalizer.transform(to_transform)
    print('d')
    



In [ ]:
scaled = get_svs_array('/data/tcia/PDA/C3L-00102-22.svs', scale=.1)
plt.imshow(scaled)

In [ ]:
img = get_svs_array('/data/tcia/PDA/C3L-00102-22.svs', scale=1.)
img.shape

In [ ]:
img

In [ ]:
s = img[14000:18000, 14000:18000]

In [ ]:
plt.imshow(s)

In [ ]:
# ref = '/home/estorrs/violet/data/he/ffpe/tcia_ccrcc/train/C3L-00165-22/C3L-00165-22_105_66.jpeg'
# # ref = '/home/estorrs/violet/data/he/ffpe/tcia_pda/train/C3L-00017-21/C3L-00017-21_100_104.jpeg'
# target = staintools.read_image(ref)
# Standardize brightness (optional, can improve the tissue mask calculation)
target = staintools.LuminosityStandardizer.standardize(s)

# Stain normalize
normalizer = staintools.StainNormalizer(method='vahadane')
normalizer.fit(target)

In [ ]:
%%time
# to_transform = staintools.read_image(fp)
to_transform = img
to_transform = staintools.LuminosityStandardizer.standardize(to_transform)
transformed = normalizer.transform(to_transform)

In [ ]:
%%time
# to_transform = staintools.read_image(fp)
to_transform = s
to_transform = staintools.LuminosityStandardizer.standardize(to_transform)
transformed = normalizer.transform(to_transform)

In [ ]:
%%time
# to_transform = staintools.read_image(fp)
to_transform = s[:150, :150]
augmentor = staintools.StainAugmentor(method='vahadane', sigma1=0.2, sigma2=0.2)
augmentor.fit(to_transform)
augmented_images = []
for _ in range(100):
    augmented_image = augmentor.pop().astype(np.uint8)
    print(np.max(augmented_image))
    plt.imshow(augmented_image)
    plt.show()
    augmented_images.append(augmented_image)

In [ ]:
transformed.shape

In [ ]:
im = Image.fromarray(s)
im.save('/home/estorrs/violet/violet/data/stain_reference.jpeg')

In [ ]:
from skimage.transform import rescale
plt.imshow(transformed)

In [ ]:
from skimage.transform import rescale
plt.imshow(transformed[14000:14100, 14000:14100])

In [ ]:
from skimage.transform import rescale
plt.imshow(transformed[:100, :100])

In [ ]:
from skimage.transform import rescale
plt.imshow(img[14000:14100, 14000:14100])

In [ ]:
plt.imshow(transformed[:100, :100])

In [ ]:
%%time
t_imgs, t_ids = extract_tif_tiles(['/data/spatial_transcriptomics/highres_images/pdac/B1-HT264P1-S1H2Fc2U1.tif'])

In [ ]:
import matplotlib.pyplot as plt
for img in t_imgs[:100]:
    plt.imshow(img)
    plt.show()

In [ ]:
%%time
s_imgs, s_ids = extract_svs_tiles({'s1': '/data/tcia/PDA/C3L-00017-22.svs'})

In [ ]:
import matplotlib.pyplot as plt
for img in s_imgs[:100]:
    plt.imshow(img)
    plt.show()

In [ ]:
for fp in fps:
    img = np.array(default_loader(fp))
    plt.imshow(img)
    plt.show()

    n = normalize_he_image(img)
    plt.imshow(n)
    plt.show()